In [1]:
!pip install transformers

In [2]:
!pip install torch

In [3]:
!nvidia-smi

Fri Nov 24 00:30:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install fugashi
!pip install mecab-python3
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=656fb0896c6d32630ef778a106be911667664baf53c322cd0198202c389b9693
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [6]:
import torch
import time
#from transformers import BertTokenizer, BertModel
from transformers import BertJapaneseTokenizer, BertModel
# Load the Tohoku's Japanese BERT model
model_name = "cl-tohoku/bert-large-japanese-v2"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Set the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print (device)

# Define a function to classify a text and get CLS's embedding
def get_cls_embedding_and_tokenized_output(input_corpus):
    # Tokenize input and get token outputs
    encoded_input_corpus  = tokenizer(input_corpus, return_tensors="pt", truncation=True)
    encoded_input_corpus = {key: val.to(device) for key, val in encoded_input_corpus.items()}

    # Run the model and get CLS's embedding
    start_time = time.time()
    with torch.no_grad():
        output  = model(**encoded_input_corpus )
        cls_embedding = output.last_hidden_state[:, 0]
        # Convert the tokenized output to a list of tokens
        tokenized_output = tokenizer.convert_ids_to_tokens(encoded_input_corpus["input_ids"].squeeze().cpu().numpy())
    elapsed_time = time.time() - start_time
    return cls_embedding, tokenized_output, elapsed_time, encoded_input_corpus

#input_corpus = "これは日本語のテキストのコープスです。"
input_corpus = """
[オペレーター] : こんにちは！お電話ありがとうございます。私の名前はエミリーで、今日はお手伝いできて嬉しいです。どのようにお手伝いできますか？
[ユーザー] : こんにちは、エミリー。貴社のサービスに興味がありますが、それが私にとって適しているかどうか分かりません。
[オペレーター] : 完全に理解していますし、興味を持っていただきありがとうございます！様々なニーズに対応した幅広いサービスを提供しています。サービスに関してもう少し教えていただけますか？
[ユーザー] : そうですね、信頼性があり、手頃な価格のものが必要です。過去にサービスで悪い経験をしました。
[オペレーター] : 了解しました。当社はすべてのサービスで信頼性と手頃な価格を重視しています。パッケージについての詳細を共有させていただきます。現在、新規顧客向けに割引料金が適用される特別なプロモーションを実施しています。これにより、高品質なサービスを手頃な価格でご利用いただけます。
[ユーザー] : 興味深いですね。サービスの具体的な機能について教えていただけますか？
[オペレーター] : もちろんです！当社のサービスには [機能1]、[機能2]、および [機能3] が含まれており、お客様の特定のニーズに応じて設計されています。これらの機能は非常に有益だと感じている現在のお客様からも優れたフィードバックをいただいています。
[ユーザー] : うーん、有望ですね。でもまだ自分に合っているかどうか分かりません。
[オペレーター] : お気持ち理解しています。迷われているのは分かります。お客様の決断をサポートするために、30日間の返金保証もご用意しています。これにより、1か月間はリスクを取らずにサービスを試していただけます。期待通りでない場合は、全額返金いたします。お気軽にお試しください。
[ユーザー]: それは寛大な提案ですね。保証について感謝します。
"""
cls_embedding, tokenized_output, elapsed_time, encoded_input_corpus = get_cls_embedding_and_tokenized_output(input_corpus)

print("CLS embedding's first 20 elements:", cls_embedding.squeeze()[:20].tolist())
print("CLS embedding size:", cls_embedding.size())
print("Inference time:", elapsed_time)
print("Input corpus: ", input_corpus)
print("Tokenized output:", tokenized_output)
print("Tokenized output length:", len(tokenized_output))
# Print encoded_input_corpus properties
print("Input IDs:", encoded_input_corpus["input_ids"])
print("Attention Mask:", encoded_input_corpus["attention_mask"])
print("Token Type IDs:", encoded_input_corpus["token_type_ids"])

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

cuda
CLS embedding's first 20 elements: [0.30973920226097107, -0.8247776627540588, 1.4520456790924072, 0.4861079752445221, -0.20203886926174164, -0.628887414932251, -0.3529073894023895, 0.2580889165401459, -0.4132477045059204, 0.24694232642650604, -0.11800813674926758, -0.49830490350723267, -1.2888479232788086, 0.05457565188407898, -0.593340277671814, -0.9605613946914673, 1.1399401426315308, 0.21966347098350525, -0.3201182782649994, -1.136432409286499]
CLS embedding size: torch.Size([1, 1024])
Inference time: 0.15050005912780762
Input corpus:  
[オペレーター] : こんにちは！お電話ありがとうございます。私の名前はエミリーで、今日はお手伝いできて嬉しいです。どのようにお手伝いできますか？
[ユーザー] : こんにちは、エミリー。貴社のサービスに興味がありますが、それが私にとって適しているかどうか分かりません。
[オペレーター] : 完全に理解していますし、興味を持っていただきありがとうございます！様々なニーズに対応した幅広いサービスを提供しています。サービスに関してもう少し教えていただけますか？
[ユーザー] : そうですね、信頼性があり、手頃な価格のものが必要です。過去にサービスで悪い経験をしました。
[オペレーター] : 了解しました。当社はすべてのサービスで信頼性と手頃な価格を重視しています。パッケージについての詳細を共有させていただきます。現在、新規顧客向けに割引料金が適用される特別なプロモーションを実施しています。これにより、高品質なサービスを手頃な価格でご利用いただけます。
[ユーザー] : 興味深いですね。サー